# Cell Averager

### Initial imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys

import numpy as np
from matplotlib import pyplot as plt

import seaborn as sns
import pandas as pd

import os

### User defined parameters

In [ ]:
ehooke_path = r"INSERT EHOOKE PATH HERE"
root_folder = r"INSERT ROOT FOLDER PATH HERE"                    
result_folder = r"INSERT RESULT FOLDER PATH HERE"

# CHANGE THE FOLLOWING VARIABLES ACCORDING TO YOUR NEEDS
base_filename = "MEMBRANE"
membrane_filename = 'MEMBRANE'
base_type = "Membrane"
dna_filename = 'DNA'


In [ ]:
sys.path.append(ehooke_path)

from images import ImageManager
from parameters import MaskParameters, RegionParameters, CellParameters, ParametersManager
from segments import SegmentsManager
from cells import CellManager
from cellcycleclassifier import CellCycleClassifier

from model import ExperimentalCondition, Replicate

You are using Python 3.6, which is deprecated and will no longer be supported in future versions of StarDist.
→ Please upgrade to Python 3.7 or later.
Using TensorFlow backend.


In [5]:
def distance(p1,p2):
    # l2 norm
    return np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

def minpointdistance(points_1,points_2):
    
    if (not points_1) or (not points_2):
        return None
    
    mind = 1e12
    for i in points_1:
        for j in points_2:
            if i!=j:
                d = distance(i,j)
                if d<mind:
                    mind=d
    if mind!=1e12:
        return mind*0.08
    else:
        return None

### This exemplifies full analysis including cell cycle classification for each condition
#### Outputs include foci counts, foci coordinates, distances between foci and their histograms, heatmaps and morphology parameters

In [ ]:
for c in os.listdir(root_folder):
    
    condition_folder = os.path.join(root_folder,c)
    
    if os.path.exists(os.path.join(result_folder,f"models_{c}.jpeg")):
        print("skipping", c)
        continue
    if not os.path.isdir(condition_folder):
        continue
        
    fluor1 = 'FOCI'
    fluor2 = 'none'
    
    print(c, fluor1, fluor2)
    
    base = "MEMBRANE"
    membrane = 'MEMBRANE'
    basetype = "Membrane"
    dnaname = 'DNA'
    pxsize = 0.08

    exp_objects = ExperimentalCondition(condition_folder, fluor1, fluor2, membrane, base, basetype, dnaname,pxsize)
    
    models_fluor_1 = exp_objects.askformodel(channel=1, modeltype='spot', minspots=1, cellcycle=(0,1,2,3))
    scatter_coords_1 = exp_objects.askforscatter(channel=1, minspots=1, maxspots=np.inf, cellcycle=(0,1,2,3))
    df_1 = exp_objects.askfortable(channel=1)
    
    if fluor2 != 'none':
        models_fluor_2 = exp_objects.askformodel(channel=2, modeltype='spot', minspots=1, cellcycle=(0,1,2,3))
        scatter_coords_2 = exp_objects.askforscatter(channel=2, minspots=1, maxspots=np.inf, cellcycle=(0,1,2,3))
        df_2 = exp_objects.askfortable(channel=2)
    else:
        models_fluor_2 = 0,0,0,np.zeros((30,30))
        scatter_coords_2 = [],[]
    
     # merge dfs
    if fluor2 != 'none':
        finaldf = pd.merge(df_1,df_2)
    else:
        finaldf = df_1
        
    # build mindistances
    mind11 = []
    mind12 = []
    mind22 = []
    for index,row in finaldf.iterrows():
        sp1 = row['SpotCoords_1']

        # 1 to 1
        mind11.append(minpointdistance(sp1,sp1))

        try:
            sp2 = row['SpotCoords_2']
            # 1 to 2
            mind12.append(minpointdistance(sp1,sp2))
            mind22.append(minpointdistance(sp2,sp2))
        except:
            pass

    finaldf['MinDistance_1to1'] = mind11

    try:
        finaldf['MinDistance_1to2'] = mind12
        finaldf['MinDistance_2to2'] = mind22
    except:
        pass
    
    # save finaldf
    finaldf.to_csv(os.path.join(result_folder,f"results_{c}.csv"))
        
    # totalN, totalCells, totalSpots, final_model
    
    fig, ax = plt.subplots(3, 2, dpi=300)
    
    ax[0,0].imshow(models_fluor_1[3], cmap='coolwarm')
    ax[0,1].imshow(models_fluor_2[3], cmap='coolwarm')
    ax[0,0].axis('off')
    ax[0,1].axis('off')
    ax[0,0].set_title(f"{fluor1}->=1 spot cells", fontsize=4)
    ax[0,1].set_title(f"{fluor2}->=1 spot cells", fontsize=4)
    
    #ax[1,0].plot(scatter_coords_1[0],scatter_coords_1[1],'x')
    #ax[1,1].plot(scatter_coords_2[0],scatter_coords_2[1],'x')
    ax[1,0].axis('off')
    ax[1,1].axis('off')
    ax[1,0].set_title(f"{fluor1}->=1 spot cells", fontsize=4)
    ax[1,1].set_title(f"{fluor2}->=1 spot cells", fontsize=4)
    
    rawtxt = f"Total cells={models_fluor_1[1]} \n Cells with at least 1 spot={models_fluor_1[0]} \n Total spots={models_fluor_1[2]}"
    ax[2,0].axis([0,10,0,10])
    ax[2,0].axis('off')
    ax[2,0].text(5,5,rawtxt, fontsize=5, wrap=True)
    
    rawtxt = f"Total cells={models_fluor_2[1]} \n Cells with at least 1 spot={models_fluor_2[0]} \n Total spots={models_fluor_2[2]}"
    ax[2,1].axis([0,10,0,10])
    ax[2,1].axis('off')
    ax[2,1].text(5,5,rawtxt, fontsize=5, wrap=True)
    
    plt.savefig(os.path.join(result_folder,f"models_{c}.jpeg"))
    plt.close('all')
    
    
    
    
    
    fig,ax = plt.subplots(1,1,dpi=300)
    sns.displot(data=df_1,x='Nspots_1',hue='CCP',ax=ax,stat='count',discrete=True,alpha=0.3,palette="bright")
    plt.savefig(os.path.join(result_folder,f"histogram1_{c}.jpeg"))
    
    fig,ax = plt.subplots(1,1,dpi=300)
    sns.displot(data=finaldf,x='MinDistance_1to1',hue='CCP',ax=ax,stat='count',discrete=False,alpha=0.3,palette="bright")
    plt.savefig(os.path.join(result_folder,f"distances1to1_{c}.jpeg"))
    
    try:
        fig,ax = plt.subplots(1,1,dpi=300)
        sns.displot(data=df_2,x='Nspots_2',hue='CCP',ax=ax,stat='count',discrete=True,alpha=0.3,palette="bright")
        plt.savefig(os.path.join(result_folder,f"histogram2_{c}.jpeg"))
        
        fig,ax = plt.subplots(1,1,dpi=300)
        sns.displot(data=finaldf,x='MinDistance_1to2',hue='CCP',ax=ax,stat='count',discrete=False,alpha=0.3,palette="bright")
        plt.savefig(os.path.join(result_folder,f"distances1to2_{c}.jpeg"))

        fig,ax = plt.subplots(1,1,dpi=300)
        sns.displot(data=finaldf,x='MinDistance_2to2',hue='CCP',ax=ax,stat='count',discrete=False,alpha=0.3,palette="bright")
        plt.savefig(os.path.join(result_folder,f"distances2to2_{c}.jpeg"))
    except:
        pass
    
    plt.close('all')
    

32_WT_ori FOCI none
46_WT_ter FOCI none
47_WT_left FOCI none
48_WT_right FOCI none
68_ftsK_ori FOCI none
69_xerC_ori FOCI none
70_xerC_ter FOCI none
71_smcSTOP_ori FOCI none
73_ftsK_ter FOCI none
74_smcSTOP_ter FOCI none
75_spoIIIE_ori FOCI none
76_noc_ori FOCI none
77_noc_ter FOCI none
78_spoIIIE_ter FOCI none
ParB_ori FOCI none
ParB_Ter FOCI none


### This exemplifies a more detailed analysis
#### Outputs are the same but now we have heatmaps done PER cell cycle phase

In [ ]:
wts = ['32_WT_ori','46_WT_ter','47_WT_left','48_WT_right']

for c in wts:
    
    condition_folder = os.path.join(root_folder,c)
    
    if os.path.exists(os.path.join(result_folder,"wts",f"models_{c}.jpeg")):
        print("skipping", c)
        continue
    if not os.path.isdir(condition_folder):
        continue
        
    fluor1 = 'FOCI'
    fluor2 = 'none'
    
    print(c, fluor1, fluor2)
    
    base = "MEMBRANE"
    membrane = 'MEMBRANE'
    basetype = "Membrane"
    dnaname = 'DNA'
    
    exp_objects = ExperimentalCondition(condition_folder, fluor1, fluor2, membrane, base, basetype, dnaname)
    
    models_fluor_1 = exp_objects.askformodel(channel=1, modeltype='spot', minspots=1, cellcycle=(1,))
    models_fluor_2 = exp_objects.askformodel(channel=1, modeltype='spot', minspots=1, cellcycle=(2,))
    models_fluor_3 = exp_objects.askformodel(channel=1, modeltype='spot', minspots=1, cellcycle=(3,))
    
    fig, ax = plt.subplots(2, 3, dpi=300)
    ax[0,0].imshow(models_fluor_1[3], cmap='coolwarm')
    ax[0,1].imshow(models_fluor_2[3], cmap='coolwarm')
    ax[0,2].imshow(models_fluor_3[3], cmap='coolwarm')
    ax[0,0].axis('off')
    ax[0,1].axis('off')
    ax[0,2].axis('off')
    ax[0,0].set_title(f"PHASE 1 ->=1 spot cells", fontsize=4)
    ax[0,1].set_title(f"PHASE 2 ->=1 spot cells", fontsize=4)
    ax[0,2].set_title(f"PHASE 3 ->=1 spot cells", fontsize=4)
    
    rawtxt = f"Total cells={models_fluor_1[1]} \n Cells with at least 1 spot={models_fluor_1[0]} \n Total spots={models_fluor_1[2]}"
    ax[1,0].axis([0,10,0,10])
    ax[1,0].axis('off')
    ax[1,0].text(5,5,rawtxt, fontsize=5, wrap=True)
    
    rawtxt = f"Total cells={models_fluor_2[1]} \n Cells with at least 1 spot={models_fluor_2[0]} \n Total spots={models_fluor_2[2]}"
    ax[1,1].axis([0,10,0,10])
    ax[1,1].axis('off')
    ax[1,1].text(5,5,rawtxt, fontsize=5, wrap=True)
    
    rawtxt = f"Total cells={models_fluor_3[1]} \n Cells with at least 1 spot={models_fluor_3[0]} \n Total spots={models_fluor_3[2]}"
    ax[1,2].axis([0,10,0,10])
    ax[1,2].axis('off')
    ax[1,2].text(5,5,rawtxt, fontsize=5, wrap=True)
    
    plt.savefig(os.path.join(result_folder,"wts",f"models_{c}.jpeg"))
    plt.close('all')


skipping 32_WT_ori
46_WT_ter FOCI none
47_WT_left FOCI none
48_WT_right FOCI none
